In [36]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader
from collections import Counter


In [37]:
spine_data = pd.read_csv('/Dataset_spine.csv')

print(spine_data.columns)


Index(['Col1', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6', 'Col7', 'Col8', 'Col9',
       'Col10', 'Col11', 'Col12', 'Class_att', 'Unnamed: 13'],
      dtype='object')


In [38]:
spine_data = spine_data.drop(columns=['Unnamed: 13'])
X = spine_data.drop(columns=['Class_att'])
y = spine_data['Class_att']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=62)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### PyTorch Dataset

In [41]:
class SpineDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [43]:
train_dataset = SpineDataset(X_train, y_train)
test_dataset = SpineDataset(X_test, y_test)
train_loader = DataLoader(dataset=train_dataset, batch_size=62, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=62, shuffle=True)


In [52]:

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Neuralnet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

### Hyperparameters

In [47]:
input_size = X_train.shape[1]
hidden_size = 60
num_classes = 2
num_epochs = 4
learning_rate = 0.002

model = NeuralNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


### Train the model

In [48]:
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        outputs = model(features)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

print("Training complete.")

Training complete.


In [49]:
def predict_spine_condition(model, new_data):
    new_data_tensor = torch.tensor(new_data.values, dtype=torch.float32)
    model.eval()
all_predictions = []

with torch.no_grad():
    for data, _ in test_loader:
        outputs = model(data)
        predicted_classes = torch.argmax(outputs, dim=1)
        all_predictions.extend(predicted_classes.cpu().numpy())


In [50]:
predictions_counter = Counter(all_predictions)

In [51]:
normal_count = predictions_counter[0]
abnormal_count = predictions_counter[1]
threshold = 0.5
print(f"Number of people classified as normal: {normal_count}")
print(f"Number of people classified as abnormal: {abnormal_count}")

Number of people classified as normal: 56
Number of people classified as abnormal: 6
